In [1]:
DATA_PATH = "Data/"

In [2]:
# FASTA file reader
from collections import OrderedDict
from typing import Dict

NAME_SYMBOL = '>'


def parse_sequences(filename: str,
                    ordered: bool=False) -> Dict[str, str]:
    """
    Parses a text file of genome sequences into a dictionary.
    Arguments:
      filename: str - The name of the file containing the genome info.
      ordered: bool - Set this to True if you want the result to be ordered.
    """
    result = OrderedDict() if ordered else {}

    last_name = None
    with open(filename) as sequences:
        for line in sequences:
            if line.startswith(NAME_SYMBOL):
                last_name = line[1:-1]
                result[last_name] = []
            else:
                result[last_name].append(line[:-1])

    for name in result:
        result[name] = ''.join(result[name])

    return result

In [3]:
# numbers txt file reader
def parse_numbers(path):
    """
    Parses a text file of numbers into a list
    """
    with open(path, 'r') as handle:
        for line in handle:
            if not line.strip():
                continue  # This skips blank lines

            values = [*map(int, line.split())]
            return values

In [4]:
# RNA codon table
rna_codon = {"UUU" : "F", "CUU" : "L", "AUU" : "I", "GUU" : "V",
           "UUC" : "F", "CUC" : "L", "AUC" : "I", "GUC" : "V",
           "UUA" : "L", "CUA" : "L", "AUA" : "I", "GUA" : "V",
           "UUG" : "L", "CUG" : "L", "AUG" : "M", "GUG" : "V",
           "UCU" : "S", "CCU" : "P", "ACU" : "T", "GCU" : "A",
           "UCC" : "S", "CCC" : "P", "ACC" : "T", "GCC" : "A",
           "UCA" : "S", "CCA" : "P", "ACA" : "T", "GCA" : "A",
           "UCG" : "S", "CCG" : "P", "ACG" : "T", "GCG" : "A",
           "UAU" : "Y", "CAU" : "H", "AAU" : "N", "GAU" : "D",
           "UAC" : "Y", "CAC" : "H", "AAC" : "N", "GAC" : "D",
           "UAA" : "STOP", "CAA" : "Q", "AAA" : "K", "GAA" : "E",
           "UAG" : "STOP", "CAG" : "Q", "AAG" : "K", "GAG" : "E",
           "UGU" : "C", "CGU" : "R", "AGU" : "S", "GGU" : "G",
           "UGC" : "C", "CGC" : "R", "AGC" : "S", "GGC" : "G",
           "UGA" : "STOP", "CGA" : "R", "AGA" : "R", "GGA" : "G",
           "UGG" : "W", "CGG" : "R", "AGG" : "R", "GGG" : "G" 
           }

### Problem 21 - Locating Restriction Sites

A DNA string is a reverse palindrome if it is equal to its reverse complement. For instance, GCATGC is a reverse palindrome because its reverse complement is GCATGC. 

Given: A DNA string of length at most 1 kbp in FASTA format.

Return: The position and length of every reverse palindrome in the string having length between 4 and 12. You may return these pairs in any order.

Sample Dataset

$\gt$Rosalind_24

TCAATGCATGCGGGTCTATATGCAT

Sample Output

4 6

5 4

6 6

7 4

17 4

18 4

20 6

21 4



### Solution 21

In [10]:
m = 'TCAATGCATGCGGGTCTATATGCAT'

In [15]:
def s21(m):
    def palindrome(string):
        #Reverse complement of a DNA string
        def replace(string):
            tr = str.maketrans(dict(zip('ACGT', 'TGCA')))
            return string.translate(tr)[::-1]
        #All substrings in DNA string
        def substrings(input_string):
            length = len(input_string)
            return [input_string[i:j+1] for i in range(length) for j in range(i,length)]
        #Finding all palindromes, only substrings with lenght between 4 and 12
        subs = list(set(substrings(string)))
        subs4_12 = []
        for i in subs:
            if len(i) >= 4 and len(i) <= 12:
                subs4_12.append(i)
        palindromes = []
        #appending all substrings who are equal to the their reverse complement
        for i in range(len(subs4_12)):
            if subs4_12[i] == replace(subs4_12[i]):
                palindromes.append(subs4_12[i])
        return palindromes
    def s21a(a, b):
        result = []
        for i in range(0, len(a) - len(b) + 1):
            if a[i:i+len(b)] == b:
                result.append(i+1)
        for i in result:
            print(str(i) + ' ' + str(len(b)))
    for i in palindrome(m):
        s21a(m, i)

In [16]:
s21(m)

5 4
21 4
4 6
20 6
7 4
18 4
17 4
6 6


In [6]:
FASTAfile = parse_sequences(DATA_PATH + 'rosalind_revp.txt')

In [17]:
s21(list(FASTAfile.values())[0])

272 4
639 4
950 4
219 4
493 4
6 4
57 4
362 4
342 6
46 6
900 6
279 4
637 4
476 4
546 4
462 6
846 6
452 6
626 6
545 6
217 8
625 8
332 4
415 4
423 4
949 6
47 4
797 4
926 4
496 8
460 10
117 4
388 4
453 4
627 4
995 4
56 6
184 4
775 4
838 4
615 6
218 6
495 10
498 4
616 4
938 4
105 4
528 4
901 4
497 6
461 8
845 8
45 8
494 12
327 4
346 4
773 4
106 4
463 4
847 4
932 4
988 4
1 4
126 4
343 4
436 4
538 4
972 4
980 4
116 6
925 6


### Problem 22 - RNA Splicing 

After identifying the exons and introns of an RNA string, we only need to delete the introns and concatenate the exons to form a new string ready for translation.

Given: A DNA string $s$ (of length at most 1 kbp) and a collection of substrings of s acting as introns. All strings are given in FASTA format.

Return: A protein string resulting from transcribing and translating the exons of $s$. 

Sample Dataset

$\gt$Rosalind_10

ATGGTCTACATAGCTGACAAACAGCACGTAGCAATCGGTCGAATCTCGAGAGGCATATGGTCACATGATCGGTCGAGCGTGTTTCAAAGTTTGCGCCTAG

$\gt$Rosalind_12

ATCGGTCGAA

$\gt$Rosalind_15

ATCGGTCGAGCGTGT

Sample Output

MVYIADKQHVASREAYGHMFKVCA


In [48]:
dic = {'string': 'ATGGTCTACATAGCTGACAAACAGCACGTAGCAATCGGTCGAATCTCGAGAGGCATATGGTCACATGATCGGTCGAGCGTGTTTCAAAGTTTGCGCCTAG',
      'intron1': 'ATCGGTCGAA',
'intron2': 'ATCGGTCGAGCGTGT'}

In [49]:
def s22(dictionary):
    string = list(dictionary.values())[0]
    introns = list(dictionary.values())[1:]
    for i in introns:
        string = string.replace(i, '')
    protein_string = ""
    for i in range(0, len(string.replace('T', 'U'))-(3+len(string.replace('T', 'U'))%3), 3):
        if rna_codon[string.replace('T', 'U')[i:i+3]] == "STOP" :
            break
        protein_string += rna_codon[string.replace('T', 'U')[i:i+3]]
    return protein_string

In [57]:
s22(dic)

'MVYIADKQHVASREAYGHMFKVCA'

In [59]:
s22(parse_sequences(DATA_PATH + 'rosalind_splc.txt'))

'MWACPSGSRMSCPPSYTSAGIFPPASTLPSTWQGFRAPLCRTSSGTRNPMSSNRLIVCHGTFRFSICSKPPYVWGRSPRLLATSILELSTSNTHTVLDTVAGNDFMANPSMHTLAPGCKCFVQNVVYLRDSTCLDKSAAARSVSCGMNSRKCGPPCALYDTFR'

### Problem 23 - Enumerating k-mers Lexicographically

Assume that an alphabet $A$ has a predetermined order; that is, we write the alphabet as a permutation $A=(a_{1},a_{2},…,a_{k})$, where $a_{1}<a_{2}<⋯<a_{k}$. For instance, the English alphabet is organized as $(A,B,…,Z)$.

Given two strings $s$ and $t$ having the same length $n$, we say that $s$ precedes $t$ in the lexicographic order (and write $s<_{Lex}t$) if the first symbol $s_{[j]}$ that doesn't match $t_{[j]}$ satisfies $s_{j}<t_{j}$ in $A$.

Given: A collection of at most 10 symbols defining an ordered alphabet, and a positive integer $n(n\leq10)$.

Return: All strings of length $n$ that can be formed from the alphabet, ordered lexicographically (use the standard order of symbols in the English alphabet).

Sample Dataset

A C G T
2

Sample Output

AA

AC

AG

AT

CA

CC

CG

CT

GA

GC

GG

GT

TA

TC

TG

TT


### Solution 23

In [81]:
import itertools

In [107]:
def s23(string, k):
    perms = []
    for p in itertools.product(string, repeat=k):
        perms.append(p)
    joiner = "".join
    for i in (list(map(joiner, perms))):
        print(i)

In [108]:
s23('ACGT', 2)

AA
AC
AG
AT
CA
CC
CG
CT
GA
GC
GG
GT
TA
TC
TG
TT


In [112]:
lines = open(DATA_PATH + 'rosalind_lexf.txt').readlines()

In [113]:
lines

['A B C D E F\n', '3\n']

In [117]:
s23(lines[0][:-1].replace(" ", ""), int(lines[1][:-1]))

### Problem 24 - Longest Increasing Subsequence 

A subsequence of a permutation is a collection of elements of the permutation in the order that they appear. For example, (5, 3, 4) is a subsequence of (5, 1, 3, 4, 2).
A subsequence is increasing if the elements of the subsequence increase, and decreasing if the elements decrease. For example, given the permutation (8, 2, 1, 6, 5, 7, 4, 3, 9), an increasing subsequence is (2, 6, 7, 9), and a decreasing subsequence is (8, 6, 5, 4, 3). You may verify that these two subsequences are as long as possible.

Given: A positive integer n$\leq$10000
followed by a permutation $\pi$ of length $n$.

Return: A longest increasing subsequence of $\pi$, followed by a longest decreasing subsequence of $\pi$.
Sample Dataset

5

5 1 4 2 3

Sample Output

1 2 3

5 4 2



### Solution 24 

In [8]:
# http://saradoesbioinformatics.blogspot.com/2016/07/longest-increasing-subsequence.html
data = []                                #This first bit reads the file which
with open(DATA_PATH + 'rosalind_lgis.txt', 'r') as f:   #contains the the length of the 
    for line in f:                       #permutation and then the permutation, 
        for nr in line.split():          #separated by spaces. The numbers are 
            data.append(int(nr))         #appended to a list as integers.
perm = data[1:]


def increasing(seq):
    P = [None] * len(seq)
    M = [None] * len(seq)

    L = 1
    M[0] = 0
    for i in range(1, len(seq)):
        lo = 0
        hi = L
        if seq[M[hi - 1]] < seq[i]:
            j = hi
        else:
            while hi - lo > 1:
                mid = (hi + lo) // 2
                if seq[M[mid - 1]] < seq[i]:
                    lo = mid
                else:
                    hi = mid

            j = lo
        P[i] = M[j - 1]
        if j == L or seq[i] < seq[M[j]]:
            M[j] = i
            L = max(L, j + 1)

    result = []
    pos = M[L - 1]
    for k in range(L):
        result.append(seq[pos])
        pos = P[pos]

    return (result[::-1])


def decreasing(seq):
    P = [None] * len(seq)
    M = [None] * len(seq)

    L = 1
    M[0] = 0
    for i in range(1, len(seq)):
        lo = 0
        hi = L
        if seq[M[hi - 1]] > seq[i]:
            j = hi
        else:
            while hi - lo > 1:
                mid = (hi + lo) // 2
                if seq[M[mid - 1]] > seq[i]:
                    lo = mid
                else:
                    hi = mid

            j = lo
        P[i] = M[j - 1]
        if j == L or seq[i] > seq[M[j]]:
            M[j] = i
            L = max(L, j + 1)

    result = []
    pos = M[L - 1]
    for k in range(L):
        result.append(seq[pos])
        pos = P[pos]

    return (result[::-1])

incr = increasing(perm)
decr = decreasing(perm)

print(*incr)
print(*decr)


148 187 254 263 280 315 395 407 421 465 472 579 621 630 651 788 821 833 954 989 1076 1230 1254 1259 1291 1311 1351 1437 1440 1441 1510 1529 1544 1605 1636 1653 1727 1833 1863 1913 2001 2089 2110 2208 2277 2279 2423 2500 2506 2529 2543 2561 2576 2581 2660 2674 2691 2701 2712 2715 2725 2752 2901 2913 3011 3044 3048 3119 3124 3139 3160 3188 3243 3270 3282 3299 3327 3408 3440 3499 3528 3569 3591 3614 3623 3678 3763 3930 4045 4096 4129 4139 4242 4252 4263 4267 4292 4349 4406 4505 4545 4598 4604 4645 4662 4677 4726 4785 4787 4846 4851 4852 4893 4918 4922 4972 4999 5180 5187 5336 5429 5563 5580 5653 5696 5704 5769 5822 5872 5926 5939 5971 6142 6212 6258 6283 6290 6304 6330 6373 6378 6454 6499 6517 6549 6620 6705 6734 6920 6956 6968 7004 7007 7016 7082 7124 7174 7395 7521 7542 7624 7731 7813 7823 7839 7912 7945 8108 8302 8311 8316 8450 8770 8807 8895 8938 9056 9118 9175 9221 9321 9390 9424 9606 9744 9878
9874 9867 9783 9759 9738 9733 9642 9641 9620 9482 9304 9140 9097 9062 8995 8842 8838 8808 

### Problem 25 - Genome Assembly as Shortest Superstring

For a collection of strings, a larger string containing every one of the smaller strings as a substring is called a superstring.

By the assumption of parsimony, a shortest possible superstring over a collection of reads serves as a candidate chromosome.

Given: At most 50 DNA strings of approximately equal length, not exceeding 1 kbp, in FASTA format (which represent reads deriving from the same strand of a single linear chromosome).

The dataset is guaranteed to satisfy the following condition: there exists a unique way to reconstruct the entire chromosome from these reads by gluing together pairs of reads that overlap by more than half their length.

Return: A shortest superstring containing all the given strings (thus corresponding to a reconstructed chromosome).

Sample Dataset

$\gt$Rosalind_56

ATTAGACCTG

$\gt$Rosalind_57

CCTGCCGGAA

$\gt$Rosalind_58

AGACCTGCCG

$\gt$Rosalind_59

GCCGGAATAC

Sample Output

ATTAGACCTGCCGGAATAC


### Solution 25

In [5]:
dic = {"Rosalind_56":
'ATTAGACCTG',
'Rosalind_57':
'CCTGCCGGAA',
'Rosalind_58':
'AGACCTGCCG',
'Rosalind_59':
'GCCGGAATAC'}

In [13]:
sequences = list(FASTAfile.values())

In [15]:
not_first = []
for i in range(len(sequences)):
    for j in range(len(sequences)):
        if i != j:
            if sequences[i][:len(sequences[i]) // 2] in sequences[j]:
                not_first.append(sequences[i])

In [27]:
 def s25(fasta):
    unique_first = True
    sequences = [fasta[sequence] for sequence in fasta]
    # find the sequence with unique first half
    not_first = []
    for i in range(len(sequences)):
        for j in range(len(sequences)):
            if i != j:
                if sequences[i][:len(sequences[i]) // 2] in sequences[j]:
                    not_first.append(sequences[i])
    if len(not_first) == len(sequences) - 1:
        for seq in sequences:
            if seq not in not_first:
                first_seq = seq
    
    if unique_first == True:
        shortest_superstring = first_seq
        sequences.remove(first_seq)
    
    # find and add the next sequence until all the sequences have been used
    while len(sequences) != 0:
        for sequence in sequences:
            if sequence[: len(seq) //2] in shortest_superstring:
                new_seq = sequence
                sequences.remove(new_seq)
                break
        stop = shortest_superstring.index(new_seq[: len(new_seq) //2])
        shortest_superstring = shortest_superstring[:stop]
        shortest_superstring += new_seq
    return shortest_superstring

In [28]:
FASTAfile = parse_sequences(DATA_PATH + 'rosalind_long.txt')

In [29]:
s25(FASTAfile)

'CTATCGTGGTGGTTTAACAGGCTACGCTGCGTACCGCAAGTGTCCATGAATGTTGCAAATTATAGCGGGACACAGAGCATAAGAGGTAAACAGTTACGCCCAAAGACGAGTAATGGAATCCTGTACTTTGTCATCTTTTGATACGGGGCCTATCGGTCCTCTGACCGCATTGGTTCACCTATAGCCGGGTCGTTCTTTTTATCCCCAGGACACGGCTCTAGGAAGAATCGAAAGGACCCCCAATGCTCGTCATAGCAGTTGCAAAAGTTACGGAAGGGTCATCACGCCTCGCGAGTGCCTGTTGGGAAAAATACTTCTGCATTCGATGTGGGTCCACCTGGAGTGAGATTATCGCACGGGGCGTCAACTCAGAAGTCACGTTCACATAATACTATGAAGATCGGTTGCACACAGACCCTTTTCCCATACCCAAACATGGTTATCTGTTCGTGGTACCGTATCCTTTTATAGGAATGTCGGGGGGAACGGGCCGGAGCAGCGACGACTTTAGTGTCCCGGACTCCTAATATCGGGCGCTTCTGTGGGATCAAATCTGTAAAACAGGCGAGGTAGGTGAATACTCCTATCTTGTATCGTTTTGAGCATAACGCGCACTACTTGGGGAACGAATCAATTGTTCTACGGCTCTACCGCCGCCTTACGGTTAGATCACGTCCCCTAGGAACCAACCCTTATACTCTAATGGGTCAAAGTTCCTTTTTTATGGTTTAAATAGCTTCACATGCGCGCATGCCTCAGAATGAAAATCAAACTGATACAGTGCCCTTCTCGGAGCCGTATGTAGCTGCGGGCGCGACGTAAACTCCCTCGTGCTAGGCGGTAACAACATCGCTCTGCTGTCCGAATCAGCTATAATCTAAACTGTACACCCCTCGTGCGTCGCGGACCGGCTTAACTTGCCAATCATGTACGGGCCGAGCTCGCTGCCATACGCATCTTGTCCTGATTAGTCCTTGCCCGTGCTGAATTCCCATGGCG

### Problem 26 - Perfect Matchings and RNA Secondary Structures

A matching in a graph $G$ is a collection of edges of $G$ for which no node belongs to more than one edge in the collection. If $G$ contains an even number of nodes (say $2n$), then a matching on $G$ is perfect if it contains $n$ edges, which is clearly the maximum possible.

First, let $K_{n}$ denote the complete graph on $2n$ labeled nodes, in which every node is connected to every other node with an edge, and let $p_{n}$ denote the total number of perfect matchings in $K_{n}$. For a given node $x$, there are $2n−1$ ways to join $x$ to the other nodes in the graph, after which point we must form a perfect matching on the remaining $2n−2$ nodes. This reasoning provides us with the recurrence relation $p_{n}=(2n−1)⋅p_{n}−1$; using the fact that $p_{1}$ is $1$, this recurrence relation implies the closed equation $p_{n}=(2n−1)(2n−3)(2n−5)⋯(3)(1)$.

Given an RNA string $s=s_{1}…s_{n}$, a bonding graph for $s$ is formed as follows. First, assign each symbol of $s$ to a node, and arrange these nodes in order around a circle, connecting them with edges called adjacency edges. Second, form all possible edges ${A, U}$ and ${C, G}$, called basepair edges; we will represent basepair edges with dashed edges.

A matching contained in the basepair edges will represent one possibility for base pairing interactions in $s$. For such a matching to exist, $s$ must have the same number of occurrences of 'A' as 'U' and the same number of occurrences of 'C' as 'G'.

Given: An RNA string $s$ of length at most 80 bp having the same number of occurrences of 'A' as 'U' and the same number of occurrences of 'C' as 'G'.

Return: The total possible number of perfect matchings of basepair edges in the bonding graph of $s$.
Sample Dataset

$\gt$Rosalind_23

AGCUAGUCAU

Sample Output

12


### Solution 26

In [119]:
s = 'AGCUAGUCAU'

In [146]:
def s26(string):
    print(factorial(string.count('U')) * factorial(string.count('G')))

In [149]:
s26(s)

12


In [147]:
FASTAfile = parse_sequences(DATA_PATH + 'rosalind_pmch.txt')

In [148]:
s26(list(FASTAfile.values())[0])

2277243837099849063333888000000


### Problem 27 - Partial Permutations

A partial permutation is an ordering of only $k$ objects taken from a collection containing $n$ objects (i.e., $k\leq n$). For example, one partial permutation of three of the first eight positive integers is given by $(5,7,2)$.

The statistic $P(n,k)$ counts the total number of partial permutations of $k$ objects that can be formed from a collection of $n$ objects. Note that $P(n,n)$ is just the number of permutations of $n$ objects, which we found to be equal to $n!=n(n−1)(n−2)⋯(3)(2)$ in “Enumerating Gene Orders”.

Given: Positive integers $n$
and $k$ such that $100\geq n\gt0$ and $10\geq k\gt0.$

Return: The total number of partial permutations $P(n,k)$, modulo $1,000,000$.

Sample Dataset

21 7

Sample Output

51200


### Solution 27

In [160]:
n = 21
k = 7

In [156]:
def s27(n, k):
    print(round((factorial(n)/factorial(n-k))%1000000))

In [161]:
s27(n, k)

51200


In [162]:
n, k = parse_numbers(DATA_PATH + 'rosalind_pper.txt')

In [163]:
s27(n, k)

926400


### Problem 28 - Introduction to Random Strings

An array is a structure containing an ordered collection of objects (numbers, strings, other arrays, etc.). We let $A[k]$ denote the $k$-th value in array A. You may like to think of an array as simply a matrix having only one row.

A random string is constructed so that the probability of choosing each subsequent symbol is based on a fixed underlying symbol frequency. GC-content offers us natural symbol frequencies for constructing random DNA strings. If the GC-content is $x$, then we set the symbol frequencies of C and G equal to $\frac{x}{2}$ and the symbol frequencies of A and T equal to $\frac{1-x}{2}$. For example, if the GC-content is 40%, then as we construct the string, the next symbol is 'G'/'C' with probability 0.2, and the next symbol is 'A'/'T' with probability 0.3.

In practice, many probabilities wind up being very small. In order to work with small probabilities, we may plug them into a function that "blows them up" for the sake of comparison. Specifically, the common logarithm of $x$ (defined for $x\gt0$ and denoted $log_{10}(x)$) is the exponent to which we must raise 10 to obtain $x$.

In the graph of the common logarithm function $y=log_{10}(x)$, we can see that the logarithm of $x$-values between 0 and 1 always winds up mapping to $y$-values between $−\infty$ and 0: $x$-values near 0 have logarithms close to $−\infty$, and $x$-values close to 1 have logarithms close to 0. Thus, we will select the common logarithm as our function to "blow up" small probability values for comparison.

Given: A DNA string $s$ of length at most 100 bp and an array $A$ containing at most 20 numbers between 0 and 1.

Return: An array $B$ having the same length as $A$ in which $B_{[k]}$ represents the common logarithm of the probability that a random string constructed with the GC-content found in $A_{[k]}$ will match $s$ exactly.

Sample Dataset

ACGATACAA

0.129 0.287 0.423 0.476 0.641 0.742 0.783

Sample Output

-5.737 -5.217 -5.263 -5.360 -5.958 -6.628 -7.009


### Solution 28

$P = (\frac{1-x}{2})^{AT} * (\frac{x}{2})^{GC}$

In [52]:
import math

In [72]:
s = 'ACGATACAA'
A = [0.129, 0.287, 0.423, 0.476, 0.641, 0.742, 0.783]

In [73]:
def s28(s, A):
    B = []
    for i in A:
        total=1.0
        for base in s:
            if base in ["G", "C"]:
                total*= i*0.5
            else:
                total*= (1-i)*0.5
        B.append(total)
    print(" ".join(str(round(math.log10(i),3)) for i in B))

In [74]:
s28(s, A)

-5.737 -5.217 -5.263 -5.36 -5.958 -6.628 -7.009


In [68]:
lines = open(DATA_PATH + 'rosalind_prob.txt').readlines()

In [69]:
s = lines[0][:-1]

In [70]:
A = list(map(float, lines[1][:-1].split()))

In [71]:
s28(s, A)

-83.188 -69.06 -66.378 -60.631 -57.46 -56.541 -55.066 -53.862 -53.028 -52.91 -52.926 -53.74 -54.118 -56.344 -57.995 -60.432 -64.272 -76.037


### Problem 29 - Enumerating Oriented Gene Orderings 

A signed permutation of length $n$ is some ordering of the positive integers ${1,2,…,n}$ in which each integer is then provided with either a positive or negative sign (for the sake of simplicity, we omit the positive sign). For example, $\pi=(5,−3,−2,1,4)$ is a signed permutation of length 5.

Given: A positive integer $n\leq6$.

Return: The total number of signed permutations of length $n$, followed by a list of all such permutations (you may list the signed permutations in any order).

Sample Dataset

2

Sample Output

8
-1 -2

-1 2

1 -2

1 2

-2 -1

-2 1

2 -1

2 1

### Solution 29

In [198]:
from itertools import product
from itertools import permutations

In [196]:
def s29(n):
    def merge(product):
        numbers, signs = product
        res = [int(signs[i] + str(number)) for i, number in enumerate(numbers)]
        return res
    
    numbers = list(permutations(range(1, n + 1)))
    signs = list(product('-+', repeat=n))

    signed_permutations = list(product(numbers, signs))
    signed_permutations = list(map(merge, signed_permutations))
    
    print(len(signed_permutations))
    for i in range(len(signed_permutations)):                                          
        print(*signed_permutations[i], sep=' ')      

In [203]:
s29(parse_numbers(DATA_PATH + 'rosalind_sign.txt')[0])

### Problem 30 - Finding a Spliced Motif 

A subsequence of a string is a collection of symbols contained in order (though not necessarily contiguously) in the string (e.g., ACG is a subsequence of TATGCTAAGATC). The indices of a subsequence are the positions in the string at which the symbols of the subsequence appear; thus, the indices of ACG in TATGCTAAGATC can be represented by $(2, 5, 9)$.

As a substring can have multiple locations, a subsequence can have multiple collections of indices, and the same index can be reused in more than one appearance of the subsequence; for example, ACG is a subsequence of AACCGGTT in 8 different ways.

Given: Two DNA strings $s$ and $t$ (each of length at most 1 kbp) in FASTA format.

Return: One collection of indices of $s$
in which the symbols of t appear as a subsequence of $s$. If multiple solutions exist, you may return any one.

Sample Dataset

$\gt$ Rosalind_14

ACGTACGTGACG

$\gt$ Rosalind_18

GTA


Sample Output

3 8 10


### Solution 30

In [34]:
def s30(s, t):
    positions = []

    i = j = 0
    while i < len(s) and j < len(t):
        if s[i] == t[j]:
            positions.append(i + 1)
            j += 1
        i += 1

    print(*positions, sep=' ')  

In [33]:
FASTA_file = parse_sequences(DATA_PATH + 'rosalind_sseq.txt')

In [35]:
s30(list(FASTA_file.values())[0], list(FASTA_file.values())[1])

2 7 13 14 17 22 25 30 31 32 33 34 35 38 42 43 50 55 56 60 68 69 72 73 78 79 80 86 93 95 104 106 108 114 117 118 120 121 122 124 129 131 132 134 144 145 154 157 159 160 161 162 166 168 170 177 178 180 181 183 186 191 192 199 202 207 211 212 214 218 229 230 241 243 247 252
